In [34]:
import helperfuncs as hp
import numpy as np
import sib_ldsc_z as ld
from scipy.optimize import minimize
from scipy.special import comb
from scipy.misc import derivative
import scipy.stats
from importlib import reload
import matplotlib.pyplot as plt
reload(ld)

<module 'sib_ldsc_z' from 'C:\\Users\\Hariharan\\Documents\\git_repos\\SNIPar\\ldsc_reg\\inferz\\sib_ldsc_z.py'>

# Defining the PDF and the Log Likelihoods

The likelihood for a SNP $i$ is:

$$
l_i = -\frac{d}{2} log (2 \pi) - \frac{1}{2} log ( |I + r_i S_i^{-1/2} V S_i^{-1/2}| ) - \frac{1}{2} z_i^T (I + r_i S_i^{-1/2} V S_i^{-1/2}) ^{-1} z_i
$$

And its derivative:

$$
\frac{dl}{dV} = r_i S^{-1/2} \Sigma_i^{-1} (\Sigma - z_i z_i^T) \Sigma_i^{-1} S^{-1/2}
$$

In [35]:
# N = 100
# S_size = int(N/2)
# S = np.array([np.array([[.5, 0], [0, .8]]),
#     np.array([[0.5, 0], [0, 0.8]])] * S_size )/N
# V = np.identity(2) * 0.5

N = 100
S = np.array([0.5/N] * N).reshape((N, 1, 1))
V = np.atleast_2d(0.5)

In [36]:
model = ld.sibreg(S = S)
model.simdata(V/N, N, simr = True)

Warning there is no value for z. Maybe consider simulating it
No value for U given. Generating a vector of ones (all SNPs weighted equally)
No value for r given. Generating a vector of ones for r
Simulated LD scores!
Effect Vectors Simulated!


In [37]:
Vin = hp.extract_upper_triangle(V)
model.neg_logll_grad(Vin * N)

[[-374.88353734]], [[50.]]


(array([[374.88353734]]), array([94.11500606]))

In [38]:
# Testing derivatives
aderiv = model._grad_ll_v(V, model.z[0, :], model.S[0], 
                 model.u[0], model.r[0], 
                 model.f)

nderiv = model._num_grad_V(V, model.z[0, :], model.S[0], 
                 model.u[0], model.r[0], 
                 model.f)

np.allclose(aderiv, nderiv)

True

In [42]:
# solving
output, result = model.solve(est_init = V) #, gradfunc = model._num_grad_V)
print(result)

[[-364.64527905]], [[0.5]]
[[-285.32376857]], [[1.5]]
[[-283.68003292]], [[1.59062045]]
[[-279.10121145]], [[2.02115307]]
[[-277.98733637]], [[2.27733069]]
[[-277.61093167]], [[2.49003582]]
[[-277.56280839]], [[2.58453512]]
[[-277.56045244]], [[2.609188]]
[[-277.56043548]], [[2.61143592]]
[[-277.56043547]], [[2.61148004]]
      fun: array([[277.56043547]])
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([-4.69943879e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 10
      nit: 9
   status: 0
  success: True
        x: array([2.61148004])


In [43]:
modll = model._log_ll(V, model.z[0, :], model.S[0], 
                 model.u[0], model.r[0], 
                 model.f)

In [44]:
V_norm = V/N
S_inv_root = hp.calc_inv_root(model.S[0])
dist = scipy.stats.multivariate_normal(mean = None,
                                      cov = np.eye(V.shape[0]) + model.r[0] * S_inv_root @ V_norm @ S_inv_root)

nlogll = dist.logpdf(model.z[0, :])
print(nlogll)

-3.913751212261724


In [46]:
modll

array([[-4.01295281]])